In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)

## 네이버 쇼핑 크롤링
- 셀레니움 이용(동적크롤링)
- 검색어 전달 후 표시되는 제품은 일부 제품
    - 스크롤바를 완전히 내려야 전체제품이 나타남
    - 자동화방식을 이용해 전체제품 표출 후 크롤링

In [6]:
driver = webdriver.Chrome('./driver/chromedriver.exe')

C:\Users\TECH4-29\AppData\Local\Temp\ipykernel_12176\3506397282.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver.exe')


In [7]:
# 웹사이트
driver.get("https://www.naver.com")
driver.implicitly_wait(10) # 10초 대기

In [8]:
# 쇼핑메뉴 클릭
driver.find_element(By.CSS_SELECTOR,'a.nav.shop').click()
time.sleep(2)

In [5]:
# #__next > div > div.pcHeader_header__tXOY4 > div > div > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > div._gnbSearch_inner_2Zksb > div > input

In [9]:
# 검색창 선택 후 클릭
search = driver.find_element(By.CSS_SELECTOR,'#__next > div > div.pcHeader_header__tXOY4 > div > div > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > div._gnbSearch_inner_2Zksb > div > input')
search.click()

In [10]:
# 검색어 입력
search.send_keys('아이폰13')

In [11]:
# 엔터키 전송
search.send_keys(Keys.ENTER)

#### 스크롤의 위치에 따라 표출 제품 갯수가 다르므로
    - 스크롤을 최대한 내려서 검색된 모든 제품을 표출 후
        - 현재 스크롤의 위치 얻어오기(자바스크립트 명령어 사용)
            - return window.scrollY
    - 크롤링 진행

In [12]:
# 스크롤 실행 전 높이
before_h = driver.execute_script('return window.scrollY')
before_h

0

In [13]:
# 무한 스크롤
# body태그 객체를 찾고 키보드의 end키를 전송
while True :
    # 현재 화면의 맨 아래로 스크롤을 내리기
    driver.find_element(By.CSS_SELECTOR,'body').send_keys(Keys.END)
    # 스크롤 후 페이지 로딩시간 대기
    time.sleep(1)
    
    # 스크롤 후 현재 위치
    after_h = driver.execute_script('return window.scrollY')
    print(after_h)
    if after_h == before_h :
        break
    before_h = after_h

1805
3885
6423
8954
10826
11004
11004


In [ ]:
# interval = 2
# prev_height = browser.execute_script("return document.body.scrollHeight")

# while True:
#     #화면 가장 아래로 스크롤 내리기
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     time.sleep(interval)

#     curr_height = browser.execute_script("return document.body.scrollHeight")
#     if curr_height == prev_height:
#         break
    
#     prev_height = curr_height

In [ ]:
# 상품정보 추출
# #content > div.style_content__xWg5l > div.list_basis > div > div:nth-child(41) > div > div > div.basicList_info_area__TWvzp

In [14]:
# 상품정보 추출 클래스명  : .basicList_info_area__TWvzp
items = driver.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")

In [ ]:
# .basicList_title__VfX3c

In [ ]:
# .price_price__LEGN7

In [18]:
#.basicList_title__VfX3c
for item in items :
    name = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c").text
    try : 
        price = item.find_element(By.CSS_SELECTOR,".price_price__LEGN7").text
    except :
        price = '판매중단'
    link = item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c > a").get_attribute('href')
    print(name,price,link)

아이폰 13 mini 자급제 128GB 미드나이트 Apple 874,000원 https://adcr.naver.com/adcr?x=ko5ddff81MwPACjoYvMjw////w==kflfX8VQ8goKDBMRNslbCSppI9gvLzuAHh3WFxyHfnjLJeDw+Na8g6OjwadkC5HhsHNm+OKo78s6ArTITx0TQ7FMJqr4rbPPjM9Cfip5iT7P9NpFhES52KpXdrx30q6AdwsbB9XURngrtnsJm+Z58MJvcpMVBQBvcHSbdkDZZYxaTklUMbzhRGC67u5QrqttzSvo6aPjiNgL9m/ChVG3TyPni3hSSeiiRZ0p+fu3WzE4HgYQnxpau9HJHO4IRCr3fFr7nDT6p+0OYWnNbzy9JENDHZP+SMf/FUrOenAeXY56RAjwS3c6NuELLveLcaFS1deqmSU7NS2IQRr1ONPGCiR6fm/dDhkAXylAZhZFFPALydL0cFXJKN1aylVmcOoDE+MNT4j8YRicjKU8JDl8y8O+hD1VJMV0Qu8C9obqIxThcUHwyr4HfR0gqdNEmnR6uK4hIqdkUK53sDZeqAyy4bASo0lHGTxLbVqlG3Zj/9uQx5TkTOIN/sCC6dyHlTnee2mhCJpXhV1mV3TjPk29mygviNmD8du1B1+a2GCKFd9bo0p7SuDe5rO/AWkA+pYyBeCmWEtMFAo6SufjKFatQ83UK2lAbBnMvPk3VLrzDsAK6Xb8WI8J6AwOo1RqR3pz34R3bYfcyac5BWGpSEU7RVEdOivE7rdOcpnkb4QKEwcQgRBMQxJ1zHzw8hwR4MFeUwbZCVv4r2km+tTPu2aEW4uI++ML1omry0vR7m3pc/xrmmrQSKV7bLdO8QDz11d7qLO+j0cXyWldJEaUgv192Y4l0s60/de39ewwQ1awCta+BuBR2ZDSKFwmgXoIVq1+8DEIv05AMbrRrKKzLLoAD2fsRxGKdHxxf8IiFhlwhQygSmkEhmvRKXgNFnYurgErvSI

Apple 아이폰 13 프로 256GB [자급제] 최저2,297,520원 https://cr.shopping.naver.com/adcr.nhn?x=DBWnNY%2FT1ZcRka6Pt67BG%2F%2F%2F%2Fw%3D%3Ds1%2BTAHnlQTsnqobR9gXwM78sbocj0u%2BKaHvGJ16O68wRN4NgkrPIjlczCBjhk2dSM7bBJ1Cm%2FsVK09Zq3phaLMTdckCktvQiFk4uJ0JOyHB4xHdGHCmgk1X8%2FXri15lranv29iozbySpMSlzARn4ISsv%2BjB2erv%2BZ%2Brrs5VHOiTQgGE7%2BZatKjf5hEf1XctkKTT%2FocsSnTqc5cndsp5Os6ZbkCnZt3PKDnGtnPfTpWXV9gLRMuJH34gK%2BdKxxm2wL%2BJKrK3wAIdlc1W0GjThbXUPg%2F6PYJW9LpCSumzPJoigK7CtQzZWLlur1%2FFge4%2BEpRfj8XFbogrgQmEnwq%2B5KNydw29MtgWojygKv%2FCpg9T0bV9SohnsJOp40F4RSK4iNmHAuB1P%2FyFj65R3q8rq1eC6UxJeah0Om0lkIOZtT9ZLxLbU57LuNzmnFLwGq45KoT9MEddwcBjht2We4Itl303qO%2FJJZ0C6p0MGfnHM5LgijriJ8H0NV5nx9%2BVr0aUV2z3EOcp3BAC1FUsgrsiVzPeJtAmQD5E%2BkKq0uS%2Bjd1WGIDf9TYDOC4LzgCehLsur5tn60w1JMbQN8YzkijGeWKRW0bHnqP98TvS%2FwD2N9%2FwwOEsHC78VBLnFZh%2FnDobeN&nvMid=29030492586&catId=50001519
Apple 아이폰 13 프로 맥스 128GB [자급제] 최저1,438,000원 https://cr.shopping.naver.com/adcr.nhn?x=uuWe%2B%2FKBGGGlnQq87z2mmf%2F%2F%2Fw%3D%3Dscrok8a5WQBRNpWyGjDWfFRYPq

Apple 아이폰 13 프로 512GB [LG U+] 최저980,000원 https://cr.shopping.naver.com/adcr.nhn?x=WYYUFJ9b8c5nluJPEz29uv%2F%2F%2Fw%3D%3DsBoW%2BJ5n%2B6VfQfYz6XjBE6MoRdqOR8vWClv0PJQRKLBRN4NgkrPIjlczCBjhk2dSMj98yjYv0NCoW76yu8ECHQDdckCktvQiFk4uJ0JOyHB4xHdGHCmgk1X8%2FXri15lranv29iozbySpMSlzARn4ISsv%2BjB2erv%2BZ%2Brrs5VHOiTQgGE7%2BZatKjf5hEf1XctkK2%2BqVNIqnn63p1JIwf1rssJbkCnZt3PKDnGtnPfTpWXV9gLRMuJH34gK%2BdKxxm2wL%2BJKrK3wAIdlc1W0GjThbXUPg%2F6PYJW9LpCSumzPJoigK7CtQzZWLlur1%2FFge4%2BEp%2FJJZxDJYZYOn%2Bcq%2BGwlbzHV7rj7GrYGFPYndo03%2FAH0ftFRTsbvvXWnc5%2F7qDPunQLlFUFVArUyqz78sCltUnS6UxJeah0Om0lkIOZtT9ZLxLbU57LuNzmnFLwGq45KohUDPniGAO4SJwOmG3ywBSjaOWZD02UcGB2wDAxZqi%2BpJm%2F1ne9lUZdiLJ5vX8PmzxIclZSyr0LroUz8XngGl7Qz%2FukRIYzBgWngyARGI6Bj6C0XXk8ySaOQ0OaLzzIxF2hGVyXKJd21uDl2S1qVt0vRRs2Tl2LVXyNQxua0eKL8GruH08DlKshetxY8KUEZb&nvMid=29030295586&catId=50000248
Apple 아이폰 13 mini 자급제 256GB 핑크 1,046,400원 https://adcr.naver.com/adcr?x=DerYT6IW1Z1N+vNjuyupy////w==kWsVIvOkdYsEZFCvnc4tj2qC6e9CwyJLMeliTqOefgJMunuYvWK4HEaq8/SIE8P

Apple 아이폰 13 프로 맥스 128GB [LG U+] 최저1,210,640원 https://cr.shopping.naver.com/adcr.nhn?x=Ex9Ioj2NY2PCBMHyH2Ijt%2F%2F%2F%2Fw%3D%3Dsd62RokT%2BiA6UpL%2BATyrGI7WcuYJA%2BT%2BdNzAMHJHWXIzrlQPyQuHoHFsppYbPEdHaTJhjuxl3UTU5bW2C5b5EKQPt2YYcf95NgMPtsjT%2FDa%2Fxg7xQx84QmXc4VagvsS%2F3nvcKSqVnG9lMuIR4ftdgAYZnlrSZr8PDPdBCvs8uWtdeJxHcKH4rYUhc5SxvGs%2FQkNm1xJaKTkFo%2FuhKZ%2F2UvSJO7jbxAC%2FhHJnJbE9pD225SET%2FmTh%2Bwr3iKnDGr9FRMyP34J4AJGzBRP5oh3tAOquKWQN4b7WstTXVRp4V%2Fd2Kx7G3krbJiQXzryuwwOqJhJz6%2FGQQrlbNxnrYlXtN57SDZFijbTH3hZXOEnUCn%2Bz1QQzCLreNp8sjPDBjStixj1OT%2BvUJUOEzwKKBwf%2FstVH7IO5mz83fSiUkSXQZVytFiBdPxTSuvYOb67FAYBAGpxy2zgNMJa1at1GSrSd0QN4u9dXHHCExaTB3By%2FzhJj%2FzaVgfXndF2fkBs0qzZTvfpF8bfclcJjAeddKIqLqoGXvUEmOH3uaTYX6h0bg2fAs0muS9qeBwvvj0NxUeFh6j%2FaM5nVf1rOXk0fVU4woWxyzynruzlo7VAQS2uIaJjVvU2Y6hEEwGxc1AZfON9sL4VRAH%2BsMDObarUodGweR8Q%3D%3D&nvMid=29030352586&catId=50000248
애플 아이폰13 미니 256GB 미개봉 새상품 자급제 통신사 가입 257,600원 https://cr.shopping.naver.com/adcr.nhn?x=9I35xjZuq5KX%2FZ5kkrTAov%2F%2F%2Fw%3D%3

[공식인증점] Apple 아이폰 13 Pro Max+ 애플케어플러스 (선택) 329,000원 https://cr.shopping.naver.com/adcr.nhn?x=Qf%2B%2FCgy7cn7u1ilOhQBkkf%2F%2F%2Fw%3D%3DsYX4RlikVO78nZoGNDtBLTXInBb5aGtbIsfpFPKfjd0KGgjNtjvRyY8TaEm3AQ5UwVBFNiF9KhBrHYiHzyQ3NEb53YphSMwQnVHQ%2FwlHnAAWM4xFIdCqFwtJ3guIuX8A%2Ba7MkE3lsDywPMfaFA3ukzsQfcp7YXkf%2Fu%2FXnNGbHLRRWR7fx5yQI121h3YIs%2BgYPxrgbkemE%2B9flQGh4zPwRSbB%2B4R%2FsVoqJUlqFSJI4gM4fV33pjq3o2OrBOypqdRk0NqOPGRaclpU1FVk%2Fz8jlo9s1c1%2ByZ7ysKZUkrwaBSNWrCRdUxvA177jl0VodmokhNUJnit2TyE8yh3GxJ0fiPNvP39ka5tIePAVQXAOkrnOYO%2BhfzX%2Ff0EO%2BwHisk7n4V36ogsAbT2GMBHUreotiQCZAkQwe9f03RLbODWzBvUbcZB%2BWpHe8Wjw5ZsRb4kPNe1GdcDC01vSQwBwWS4I03nSUOjvRbnLUBSaHUDq48AtbaC3fLLb64pEiFci74K%2BTK%2F8ye%2B%2BP0QNCGyKFBhyhpNSW2TeU4ypKFSOrU0NlNQ6b8HOx3t1XsFm%2FF8qU77%2BLf8WyF8XuBPumBqJGOrWoIzzUaLji1QUfW5FQpucBI9CGAv2xA9xguVkCJJpofd3uFv3tGyysaB%2ByYJy%2F%2FSuLnxuH9507WDBvHrc2pCiALOgOEsHC78VBLnFZh%2FnDobeN&nvMid=37547223025&catId=50001519
아이폰13 미니 256G 애플 546,700원 https://cr.shopping.naver.com/adcr.nhn?x=bFsnd15Absy

### 수집 후 DF로 저장

In [19]:
name = []
price = []
link = []

In [20]:
# 전체 상품 정보 상품별로 추출
items = driver.find_elements(By.CSS_SELECTOR,".basicList_info_area__TWvzp")

In [ ]:
# .price_price__LEGN7

In [21]:
for item in items :
    name.append(item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c").text)
    try : 
        price.append(item.find_element(By.CSS_SELECTOR,".price_price__LEGN7").text)
    except :
        price.append('판매중단')
    link.append(item.find_element(By.CSS_SELECTOR,".basicList_title__VfX3c > a").get_attribute('href'))


In [22]:
len(name),len(price),len(link)

(46, 46, 46)

In [23]:
pd.DataFrame({'품명':name,'가격':price,'link':link})

,품명,가격,link
0,아이폰 13 mini 자급제 128GB 미드나이트 Apple,"874,000원",https://adcr.naver.com/adcr?x=ko5ddff81MwPACjo...
1,아이폰 13 mini 자급제 128GB 핑크 Apple,"874,000원",https://adcr.naver.com/adcr?x=iTensWoT/A8Fa8vr...
2,아이폰 13 mini 자급제 256GB 스타라이트 Apple,"997,350원",https://adcr.naver.com/adcr?x=n2XbEfiWYe6NK94Y...
3,Apple 아이폰 13 256GB [자급제],"최저1,168,480원",https://cr.shopping.naver.com/adcr.nhn?x=XZXjD...
4,Apple 아이폰 13 미니 128GB [자급제],"최저851,865원",https://cr.shopping.naver.com/adcr.nhn?x=EOLJE...
5,Apple 아이폰 13 미니 256GB [자급제],"최저997,350원",https://cr.shopping.naver.com/adcr.nhn?x=Yp%2B...
6,Apple 아이폰 13 128GB [자급제],"최저1,057,300원",https://cr.shopping.naver.com/adcr.nhn?x=YImcA...
7,Apple 아이폰 13 미니 512GB [자급제],"최저1,224,000원",https://cr.shopping.naver.com/adcr.nhn?x=RVWgm...
8,Apple 아이폰 13 프로 128GB [자급제],"최저1,649,990원",https://cr.shopping.naver.com/adcr.nhn?x=lcnmi...
9,Apple 아이폰 13 프로 512GB [자급제],"최저1,649,990원",https://cr.shopping.naver.com/adcr.nhn?x=Oldny...
